<a href="https://colab.research.google.com/github/choyoona/AI-IoT/blob/main/material/deep_learning/train_yolov3_cup_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv3 커스텀 데이터 학습

copy from https://articlearn.id/article/e5571bb0-how-to-train-yolov3-on-google-colab-to-de/

# 임포트

In [ ]:
import cv2
import matplotlib.pyplot as plt
import glob


# Darknet 다운받아서 컴파일

## 프로젝트 다운로드

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git  # 다크넷 프로젝트 다운

Cloning into 'darknet'...
remote: Enumerating objects: 15460, done.
remote: Total 15460 (delta 0), reused 0 (delta 0), pack-reused 15460
Receiving objects: 100% (15460/15460), 14.05 MiB | 17.36 MiB/s, done.
Resolving deltas: 100% (10388/10388), done.


OpenCV, GPU, CuDNN 사용 관련 설정하고 컴파일

In [ ]:
%cd darknet  # 다크넷으로 이동해서

/content/darknet


## 컴파일 설정 파일 수정

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile        # option을 적어둔 것 

## 컴파일

In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old ==

# 데이터 준비
- darknet/data/obj에 압축 풀기
- classes.names와 training.data 파일 생성
- training.txt 파일 생성


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!cp /gdrive/MyDrive/tmp/cup.zip ./


## 압축 풀기

In [ ]:
!rm -rf data/obj
!mkdir data/obj
!unzip cup.zip -d data/obj

Archive:  cup.zip
   creating: data/obj/cup/
  inflating: data/obj/__MACOSX/._cup  
  inflating: data/obj/cup/image29.txt  
  inflating: data/obj/__MACOSX/cup/._image29.txt  
  inflating: data/obj/cup/image15.txt  
  inflating: data/obj/__MACOSX/cup/._image15.txt  
  inflating: data/obj/cup/image14.txt  
  inflating: data/obj/__MACOSX/cup/._image14.txt  
  inflating: data/obj/cup/image28.txt  
  inflating: data/obj/__MACOSX/cup/._image28.txt  
  inflating: data/obj/cup/image16.txt  
  inflating: data/obj/__MACOSX/cup/._image16.txt  
  inflating: data/obj/cup/image48.jpg  
  inflating: data/obj/__MACOSX/cup/._image48.jpg  
  inflating: data/obj/cup/image49.jpg  
  inflating: data/obj/__MACOSX/cup/._image49.jpg  
  inflating: data/obj/cup/image17.txt  
  inflating: data/obj/__MACOSX/cup/._image17.txt  
  inflating: data/obj/cup/image13.txt  
  inflating: data/obj/__MACOSX/cup/._image13.txt  
  inflating: data/obj/cup/image8.jpg  
  inflating: data/obj/__MACOSX/cup/._image8.jpg  
  inflat

## 학습 위한 설정 파일 생성

### data/obj.names

In [ ]:
%%writefile data/obj.names
cup

Writing data/obj.names


In [ ]:
!cat data/obj.names

cup

In [ ]:
!cp data/obj.names data/coco.names

### data/obj.data

In [ ]:
%%writefile data/obj.data
classes = 1
train = data/train.txt
valid = data/test.txt
names = /content/darknet/data/obj.names
backup = ./

Writing data/obj.data


In [ ]:
!cat data/obj.data

classes = 1
train = data/train.txt
valid = data/test.txt
names = /content/darknet/data/obj.names
backup = ./

### data/train.txt


In [ ]:
!ls -al data/obj/cup

total 50916
drwxr-xr-x 2 root root    4096 Jul 28 00:26 .
drwxr-xr-x 4 root root    4096 Jul 28 01:03 ..
-rw-r--r-- 1 root root  992630 Jul 28 00:05 image10.jpg
-rw-rw-r-- 1 root root      37 Jul 27 15:26 image10.txt
-rw-r--r-- 1 root root  987278 Jul 28 00:05 image11.jpg
-rw-rw-r-- 1 root root      37 Jul 27 15:26 image11.txt
-rw-r--r-- 1 root root 1012808 Jul 28 00:05 image12.jpg
-rw-rw-r-- 1 root root      37 Jul 27 15:26 image12.txt
-rw-r--r-- 1 root root 1039308 Jul 28 00:06 image13.jpg
-rw-rw-r-- 1 root root      37 Jul 27 15:26 image13.txt
-rw-r--r-- 1 root root 1039825 Jul 28 00:06 image14.jpg
-rw-rw-r-- 1 root root      37 Jul 27 15:26 image14.txt
-rw-r--r-- 1 root root 1038961 Jul 28 00:06 image15.jpg
-rw-rw-r-- 1 root root      37 Jul 27 15:26 image15.txt
-rw-r--r-- 1 root root 1014113 Jul 28 00:06 image16.jpg
-rw-rw-r-- 1 root root      37 Jul 27 15:26 image16.txt
-rw-r--r-- 1 root root 1017224 Jul 28 00:06 image17.jpg
-rw-rw-r-- 1 root root      37 Jul 27 15:26 image17.txt

In [ ]:
# ls -al data/obj/Raccoon 밑에 jpg 파일들이 있으면
# images_list = glob.glob("data/obj/laptops_labeled/*[jpg|png|jpeg]")
images_list = glob.glob("data/obj/cup/*[jpg|png|jpeg]")

# ls -al data/obj 밑에 jpg 파일들이 있으면
# images_list = glob.glob("data/obj/*[jpg|png|jpeg]")

print(images_list)

# Create train.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

['data/obj/cup/image36.jpg', 'data/obj/cup/image16.jpg', 'data/obj/cup/image10.jpg', 'data/obj/cup/image30.jpg', 'data/obj/cup/image28.jpg', 'data/obj/cup/image25.jpg', 'data/obj/cup/image34.jpg', 'data/obj/cup/image17.jpg', 'data/obj/cup/image38.jpg', 'data/obj/cup/image19.jpg', 'data/obj/cup/image6.jpg', 'data/obj/cup/image22.jpg', 'data/obj/cup/image9.jpg', 'data/obj/cup/image37.jpg', 'data/obj/cup/image2.jpg', 'data/obj/cup/image41.jpg', 'data/obj/cup/image44.jpg', 'data/obj/cup/image20.jpg', 'data/obj/cup/image21.jpg', 'data/obj/cup/image35.jpg', 'data/obj/cup/image11.jpg', 'data/obj/cup/image12.jpg', 'data/obj/cup/image31.jpg', 'data/obj/cup/image45.jpg', 'data/obj/cup/image14.jpg', 'data/obj/cup/image46.jpg', 'data/obj/cup/image23.jpg', 'data/obj/cup/image33.jpg', 'data/obj/cup/image4.jpg', 'data/obj/cup/image27.jpg', 'data/obj/cup/image49.jpg', 'data/obj/cup/image1.jpg', 'data/obj/cup/image43.jpg', 'data/obj/cup/image39.jpg', 'data/obj/cup/image47.jpg', 'data/obj/cup/image5.jpg

In [ ]:
!cat data/train.txt

data/obj/cup/image36.jpg
data/obj/cup/image16.jpg
data/obj/cup/image10.jpg
data/obj/cup/image30.jpg
data/obj/cup/image28.jpg
data/obj/cup/image25.jpg
data/obj/cup/image34.jpg
data/obj/cup/image17.jpg
data/obj/cup/image38.jpg
data/obj/cup/image19.jpg
data/obj/cup/image6.jpg
data/obj/cup/image22.jpg
data/obj/cup/image9.jpg
data/obj/cup/image37.jpg
data/obj/cup/image2.jpg
data/obj/cup/image41.jpg
data/obj/cup/image44.jpg
data/obj/cup/image20.jpg
data/obj/cup/image21.jpg
data/obj/cup/image35.jpg
data/obj/cup/image11.jpg
data/obj/cup/image12.jpg
data/obj/cup/image31.jpg
data/obj/cup/image45.jpg
data/obj/cup/image14.jpg
data/obj/cup/image46.jpg
data/obj/cup/image23.jpg
data/obj/cup/image33.jpg
data/obj/cup/image4.jpg
data/obj/cup/image27.jpg
data/obj/cup/image49.jpg
data/obj/cup/image1.jpg
data/obj/cup/image43.jpg
data/obj/cup/image39.jpg
data/obj/cup/image47.jpg
data/obj/cup/image5.jpg
data/obj/cup/image26.jpg
data/obj/cup/image24.jpg
data/obj/cup/image13.jpg
data/obj/cup/image3.jpg
data/ob

# 4. 사전 학습된 YOLOv3 모델 파일 다운로드

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-07-28 01:33:39--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  18.4MB/s    in 9.4s    

2022-07-28 01:33:49 (16.5 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



# 5. 학습관련 설정 파일 수정

원 설정파일 yolov3.cfg를 yolv3_custom.cfg로 복사하고

복사한 설정파일을 수정

In [ ]:
# 탐지 대상 클래스 수를 설정
CLASS_NUM = 1

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

classes = CLASS_NUM
filters = (classes + 5) * 3
# max_batches = classes * 2000
max_batches = classes * 1000
steps1 = int(max_batches*0.8)
steps2 = int(max_batches*0.9)
# if max_batches < 6000:
#   max_batches = 6000

# Edit classes & filters
!sed -i 's/batch=1/batch=64/' cfg/yolov3_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = $max_batches/' cfg/yolov3_custom.cfg
!sed -i '610 s@classes=80@classes=$classes@' cfg/yolov3_custom.cfg
!sed -i '696 s@classes=80@classes=$classes@' cfg/yolov3_custom.cfg
!sed -i '783 s@classes=80@classes=$classes@' cfg/yolov3_custom.cfg
!sed -i '603 s@filters=255@filters=$filters@' cfg/yolov3_custom.cfg
!sed -i '689 s@filters=255@filters=$filters@' cfg/yolov3_custom.cfg
!sed -i '776 s@filters=255@filters=$filters@' cfg/yolov3_custom.cfg
!sed -i 's/steps=400000,450000/steps=$steps1,$steps2/' cfg/yolov3_custom.cfg

# 6. 학습 실행

학습된 모델은  ./yolov3_custom_last.weights 로 저장된다.

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show 
# !./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.552001), count: 4, class_loss = 0.798276, iou_loss = 0.799063, total_loss = 1.597339 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000853, iou_loss = 0.000000, total_loss = 0.000853 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000605, iou_loss = 0.000000, total_loss = 0.000605 
 total_bbox = 30080, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.461486), count: 4, class_loss = 0.851904, iou_loss = 1.271202, total_loss = 2.123106 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000844, iou_loss = 0.000000, total_loss = 0.000844 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Av

## 7. 탐지 실행

In [ ]:
!./darknet detect cfg/yolov3_custom.cfg ./yolov3_custom_last.weights data/obj/cup/image1.jpg

In [ ]:
pred = plt.imread('predictions.jpg')
plt.figure(figsize=(20,20))
plt.imshow(pred)
plt.show()

In [ ]:
!cp /gdrive/MyDrive/tmp/yolov3_custom_last.weights 
# yolov3_custom_last.weights copy 하는 것 